In [1]:
import pandas as pd

In [78]:
train_pre = pd.read_csv('train.csv',header=None)
final = pd.read_csv('test.csv',header=None)
trainLabels_pre = pd.read_csv('trainLabels.csv',header=None)

In [75]:
# Some analysis

print("Are the labels balanced?")
print((trainLabels_pre.iloc[:,0]).value_counts())


Are the labels balanced?
1    510
0    490
Name: 0, dtype: int64


In [117]:
# Selecting test + training set
from random import random
test_select = pd.Series([True  if random()<0.2 else False for x in range(len(train_pre))])

train, test = train_pre[~test_select], train_pre[test_select]
trainLabels, testLabels = trainLabels_pre[~test_select], trainLabels_pre[test_select]

In [47]:
train.shape, test.shape

((810, 40), (190, 40))

# Classifiers

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [49]:
lr = LogisticRegression(solver='liblinear').fit(train,trainLabels.to_numpy().ravel())

In [50]:
accuracy_score(testLabels,lr.predict(test))

0.8315789473684211

In [52]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100).fit(train,trainLabels.to_numpy().ravel())

In [53]:
accuracy_score(testLabels,rf.predict(test))

0.8789473684210526

In [62]:
from sklearn.svm import SVC
svc = SVC(gamma='auto').fit(train,trainLabels.to_numpy().ravel())

In [63]:
accuracy_score(testLabels,svc.predict(test))

0.9631578947368421

In [94]:
## Best score 0.90658 / 0.90797

# Clustering + Classifiers

In [98]:
from sklearn.decomposition import PCA

In [108]:
pca = PCA(n_components=0.95, svd_solver='full')

train_reduced = pca.fit_transform(train)
test_reduced = pca.transform(test)

In [109]:
svc = SVC(gamma='auto').fit(train_reduced,trainLabels.to_numpy().ravel())

In [110]:
accuracy_score(testLabels,svc.predict(test_reduced))

0.9578947368421052

In [112]:
predictions = svc.predict(pca.transform(final))

In [ ]:
## Best score 0.90658 / 0.90797

# Classifiers + grid search

In [118]:
from sklearn.model_selection import GridSearchCV

In [141]:
# param grid

param_grid = {
    'n_estimators': [10,20,50,100,200,500],
    'max_depth' : [None,100,50,20,10,5],
    'criterion' : ['gini','entropy'],
    #'min_samples_split' : np.linspace(0.1,1.0,num=10),
    #'min_samples_leaf' : np.linspace(0.1,0.5,num=5),
    #'max_features' : np.linspace(0.5,1.0,num=5+1)
}

rf = RandomForestClassifier()

In [142]:
gscv = GridSearchCV(rf,param_grid=param_grid,scoring='accuracy',cv=10,n_jobs=-1,verbose=10)

In [143]:
import numpy as np
result = gscv.fit(X=train_pre,y=np.asarray(trainLabels_pre).ravel())
result.best_params_

KeyboardInterrupt: 

# Submit to Kaggle

In [113]:
file_name = "solution_3.csv"
message = "3nd attempt using SVC and PCA"
header = ['Id','Solution']



pd.DataFrame(
    data=list(zip([x for x in range(1,len(final)+1)], predictions.tolist()))
).to_csv('{}'.format(file_name), index=False, header=header)

In [114]:
%%bash -s "$file_name" "$message"
kaggle competitions submit -c data-science-london-scikit-learn -f $1 -m "$2"

Successfully submitted to Data Science London + Scikit-learn

100%|██████████| 60.5k/60.5k [00:00<00:00, 613kB/s]
